## Summary 

- Goal: Determine what caused the dip at the end of the chart shown above and, if appropriate, recommending solutions for the problem.

- Hypothesis to test: 
        a) Less engagement: people do not like the product anymore
        b) Seasonal dip 
        c) 'Wrong' measuring of the data

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
#path = '/Users/danielvalle/Documents/Coding/Insight/Data_challenges/Customer_cliff_dive/Data/'

In [4]:
users = pd.read_csv('Data/yammer_users.csv')
events = pd.read_csv('Data/yammer_events.csv') 
emails = pd.read_csv('Data/yammer_emails.csv')

In [5]:
data = pd.merge(events, users, on='user_id', how='left')
data.head()

,user_id,occurred_at,event_type,event_name,location,device,user_type,created_at,company_id,language,activated_at,state
0,10522.0,2014-05-02 11:02:39,engagement,login,Japan,dell inspiron notebook,3.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
1,10522.0,2014-05-02 11:02:53,engagement,home_page,Japan,dell inspiron notebook,3.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
2,10522.0,2014-05-02 11:03:28,engagement,like_message,Japan,dell inspiron notebook,3.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
3,10522.0,2014-05-02 11:04:09,engagement,view_inbox,Japan,dell inspiron notebook,3.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
4,10522.0,2014-05-02 11:03:16,engagement,search_run,Japan,dell inspiron notebook,3.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active


In [6]:
# Convert date columns to datetime type
data['occurred_at'] = pd.to_datetime(data['occurred_at'])

In [7]:
# Create week, start and end columns
data['occurred_period'] = data['occurred_at'].dt.to_period(freq='W')
data['occurred_start'] = data['occurred_period'].dt.start_time
data['occurred_end'] =  data['occurred_period'].dt.end_time

In [8]:
# Delete non-engaged users
engaged = data[data['event_type'] == 'engagement']
engaged_group = engaged.groupby('occurred_start')['user_id'].nunique().to_frame().reset_index()
engaged_group.head()

In [ ]:
# Convert date columns to datetime type
data['created_at'] = pd.to_datetime(data['created_at'])
data['activated_at'] = pd.to_datetime(data['activated_at'])
data['aoccurred_at_x'] = pd.to_datetime(data['occurred_at_x'])
data['occurred_at_y'] = pd.to_datetime(data['occurred_at_y'])

In [ ]:
# Only look at event_type == engagement
data['event_type'].unique()

In [ ]:
data['event_type'].value_counts()

In [ ]:
data = data[data['event_type'] == 'engagement']

In [ ]:
print(len(data))
print(data['event_type'].value_counts())

In [ ]:
data['occurred_at_x'][:20]

In [ ]:
# Set index to occured_at event
data.set_index('occurred_at_x', inplace=True)

In [ ]:
data.head()